**HyperParameter Tuning**

각 모델을 summary하는 과정에서, batch_size를 각자 다르게 해서 평가를 해본다.
batch_size의 수에 따른 정확도(learning_rate)를 평가하고, 어느 batch_size일 때 가장 수치가 괜찮은지 평가할 수 있다.

랜덤서치 - 최솟값 ~ 최댓값 사이에서 값 랜덤으로 가져와, batch_size도 1 ~ 10사이에서 랜덤으로 뽑아, learning_rate도 랜덤으로 뿌려와 실험.

learning_rate가 0.025부터 0.05, 0.1, 0.2.... 로 하고, batch_size를 1 2 4.. 로 하면 nm개의 칸을 만들어야 하는데, 이를 일일이 하면 시간이 걸린다.
어떻게 해야 효율적으로 할 수 있을까?

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

with open('X.npy', 'rb') as f:
    X = np.load(f)

with open('y.npy', 'rb') as f:
    y = np.load(f)

In [6]:
#K-fold cross validation : Data가 작을 때 validation accuracy를 믿고자 하면

#초기는, X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, randoma_state =42)
#처럼, test_size를 0.1로 10% 잘라서 테스트를 하였다
X.shape, y.shape

x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

In [7]:
# K-fold cross validation
K = 10
kf = KFold(n_splits=K)

In [8]:
rmses = []
for train_index, test_index in kf.split(scaled_X):
  scaled_X_train, scaled_X_test = scaled_X[train_index], scaled_X[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  # training
  model = keras.Sequential(
      [
          keras.Input(shape=scaled_X_train.shape[-1]),
          layers.Dense(96, activation='relu'),
          layers.Dense(48, activation='relu'),
          layers.Dense(1)
      ]
  )

  opt = keras.optimizers.Adam(learning_rate=0.005)
  model.compile(loss="mse", optimizer=opt)

  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
  model.fit(scaled_X_train, scaled_y_train, 
            batch_size=2, epochs=150, 
            callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

  # evaluation
  pred = model.predict(scaled_X_test).reshape((-1, 1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

  print(rmse)
  print("---------------------")
  
  rmses.append(rmse)
  break

print("average rmse:", np.mean(rmses))

Epoch 1/150
624/624 [==============================] - 2s 2ms/step - loss: 0.0067 - val_loss: 0.0024
Epoch 2/150
624/624 [==============================] - 1s 2ms/step - loss: 0.0046 - val_loss: 0.0023
Epoch 3/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0033 - val_loss: 0.0013
Epoch 4/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0041 - val_loss: 0.0021
Epoch 5/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0034 - val_loss: 0.0015
Epoch 6/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0034 - val_loss: 0.0022
Epoch 7/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0031 - val_loss: 0.0017
Epoch 8/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0028 - val_loss: 0.0016
Epoch 9/150
624/624 [==============================] - 1s 2ms/step - loss: 0.0032 - val_loss: 0.0015
Epoch 10/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0024 - val_lo

In [9]:
# Grid Search
# batch size와 learning rate를 설정해보낟
batch_sizes = np.arange(1, 10, 2)
learning_rates = [0.005, 0.01, 0.02]

for batch_size in batch_sizes :
  for learning_rate in learning_rates:
    print("batch size: ", batch_size, "learning rate: ", learning_rate)

batch size:  1 learning rate:  0.005
batch size:  1 learning rate:  0.01
batch size:  1 learning rate:  0.02
batch size:  3 learning rate:  0.005
batch size:  3 learning rate:  0.01
batch size:  3 learning rate:  0.02
batch size:  5 learning rate:  0.005
batch size:  5 learning rate:  0.01
batch size:  5 learning rate:  0.02
batch size:  7 learning rate:  0.005
batch size:  7 learning rate:  0.01
batch size:  7 learning rate:  0.02
batch size:  9 learning rate:  0.005
batch size:  9 learning rate:  0.01
batch size:  9 learning rate:  0.02


In [12]:
results = []

for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    print(batch_size, learning_rate)
    for train_index, test_index in kf.split(scaled_X):
      scaled_X_train, scaled_X_test = scaled_X[train_index], scaled_X[test_index]
      scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
      y_test = y[test_index]

      # training
      model = keras.Sequential(
          [
              layers.InputLayer(input_shape=scaled_X_train.shape[-1]),
              layers.Dense(96, activation='relu'), # (0.1439302, 0.123, 0.999) --> (23, 0, 255)
              layers.Dense(48, activation='relu'),
              layers.Dense(1)
          ]
      )

      #model.compile(loss="mse", optimizer="adam")

      opt = keras.optimizers.Adam(learning_rate=0.005)
      model.compile(loss="mse", optimizer=opt)

      early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
      model.fit(scaled_X_train, scaled_y_train, 
                batch_size=1, epochs=150, 
                callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

      # evaluation
      pred = model.predict(scaled_X_test).reshape((-1, 1))
      pred = y_min_max_scaler.inverse_transform(pred)
      rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

      print(rmse)
      print("---------------------")

      #results를 딕셔너리 형식으로 설정
      #batchsize와 learning_rate, rmse를 각각 저장토록 함
      result = {}
      result['batch_size'] = batch_size
      result['learning_rate'] = learning_rate
      result['rmse'] = rmse

      results.append(result)
      #fianl_rmses.append(rmse)
      break

    #break  #일단 한 세트에 대해서만 학습해보자. batchsize 1, learnig_size 0.005를 실행시 rmse 등장
  break

1 0.005
Epoch 1/150
1248/1248 [==============================] - 2s 2ms/step - loss: 0.0073 - val_loss: 0.0017
Epoch 2/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0046 - val_loss: 0.0020
Epoch 3/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0055 - val_loss: 0.0024
Epoch 4/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0045 - val_loss: 0.0027
Epoch 5/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 6/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0029 - val_loss: 0.0020
Epoch 7/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0032 - val_loss: 0.0023
Epoch 8/150
1248/1248 [==============================] - 2s 1ms/step - loss: 0.0025 - val_loss: 0.0026
35982.202282662605
---------------------
1 0.01
Epoch 1/150
1248/1248 [==============================] - 2s 2ms/step - loss: 0.0066 - val_loss: 0.0027
Epoch 2/150
1248/

In [13]:
results 
#batch는 1, learningrate는 0.01에서 가장 좋았다.
#따라서, 다음번에 돌릴 땐 이 값 근처에서 촘촘히 돌리면 된다.

[{'batch_size': 1, 'learning_rate': 0.005, 'rmse': 35982.202282662605},
 {'batch_size': 1, 'learning_rate': 0.01, 'rmse': 21973.41370919617},
 {'batch_size': 1, 'learning_rate': 0.02, 'rmse': 28741.543892787908}]

In [ ]:
"""
Random search
"""
import random
batch_sizes = random.sample(range(1, 11), 5)
leraning_rates = np.random.uniform(low = 0.005, high = 0.1, size = (4,))
#자원이 충분하다면, 위의 랜덤서치를 이용해 해결도 가능